# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

last summer when i was walking on the street i my hometown suddenly number of banks branch around me attract my attention
somthing i had never note before...
so take a look deeper around myself and i found unordinary number of banks in other streets too...
i thought that how can i find real number of these banks all around this city then i decide to use map. 
**i claimed that these number of banks is more than number of restaurants**
so this is unusual because number of cafes and restaurants is more than number of banks in each city of the world's countries!our country suffering a huge economical crisis and unreasonable number of banks in a small citie in this crisis can play an important role in consume valuable financial resources.so you can see how lack of management in banking system that we can see signs of that in number of unecessary bank branches in a small city like arak can influence whole economic in a country...
**i will try to prove this claim in this project by using maps and locations**

**in Iran,Arak bank branches disribution is so irregular.**

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* ditribution of bank branches on map can help us answere is these branches useless or not?
* number of bank branches in each streets to visualize this destribution
* number of branches of each bank to see is necessary that each bank has several branches in a small city with population about 484212 (all population include adults).[[4]](#rf4).whereas global standard is 12.5 bank branches for every 100,000 adults in 2016.[[3]](#rf3)

Following data sources will be needed to extract/generate the required information:
* foursquare json data to extract restaurants data and reverse location that found through google map into addresses[[5]](#rf5)
* address of banks and number of them that downloaded from [[1]](#rf1)

### import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import requests # library to handle requests
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# data

**data downloaded from a xls file in  https://www.cbi.ir/BanksInstitutions/BankInstitute.aspx**.
<div>and convert it to a csv file with name of 'Report.csv' in www.zamzar.com because i couldnt figure out xls file with python libraries</div>

In [4]:
ndf = pd.read_csv('Report.csv')

**now i create a new column with first part of addresses because addresses are uncertainly and too long so i decided to use first part of them then remove space between**

In [5]:
ndf['آدرس'].value_counts()

خیابان شهید بهشتی                                                   3
خیابان شهید شیرودی                                                  2
خیابان امام خمینی، چهارراه دکتر حسابی                               2
خیابان مشهد                                                         2
ميدان انقلاب                                                        2
                                                                   ..
شهرك علي بن ابيطالب ع - خیابان  سی متری - نبش خیابان فتح شیاکوه     1
کمربندی شمالی، میدان میوه و تره بار جدید، پلاک 4735                 1
خیابان دانشگاه، روبروی خیابان قیام، پلاک 47143                      1
خیابان شیخ فضل اله نوری، پلاک 7449                                  1
خیابان مشهد، پلاک 3443                                              1
Name: آدرس, Length: 204, dtype: int64

In [6]:
ndf['آدرس'].replace('،',',')

0              خيابان امام خميني، روبروي 12متري ميراشرفي
1                                       خیابان امیر کبیر
2                خیابان قائم مقام فراهانی، نبش کوچه زرین
3        خیابان شهید دکتر بهشتی، بالاتر از میدان هفت تیر
4      خیابان میرزای شیرازی، نرسیده به پل برق، ساختما...
                             ...                        
207               خیابان محسنی، نبش گذر مسجد حاج تقی خان
208                                           میدان شهدا
209                    خیابان شهید رجایی، نبش کوچه ابطحی
210                    خیابان امام، نبش کوچه حاج علیخانی
211                                       خیابان 22 بهمن
Name: آدرس, Length: 212, dtype: object

In [7]:
ndf['address'] = ndf['آدرس'].apply(lambda  آدرس: آدرس.split('،')[0])

In [8]:
ndf.tail(5)

,نام,کد شعبه,نوع فعالیت,کد اختصاصی,بانک,استان,شهر,آدرس,تلفن,نوع (شعبه/باجه),address
207,مرکزی اراک,3701,ریالی,7303701,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,خیابان محسنی، نبش گذر مسجد حاج تقی خان,08632215934,شعبه,خیابان محسنی
208,شهدا,3702,ریالی,7303702,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,میدان شهدا,08632236623,شعبه,میدان شهدا
209,مرکزی- اراک,3912,ریالی,8003912,موسسه اعتباری غیربانکی نور,مرکزی,اراک,خیابان شهید رجایی، نبش کوچه ابطحی,08632239866,شعبه,خیابان شهید رجایی
210,امام خمینی- اراک,3913,ریالی,8003913,موسسه اعتباری غیربانکی نور,مرکزی,اراک,خیابان امام، نبش کوچه حاج علیخانی,08632218320,شعبه,خیابان امام
211,قائم مقام- اراک,3916,ریالی,8003916,موسسه اعتباری غیربانکی نور,مرکزی,اراک,خیابان 22 بهمن,08632247431,شعبه,خیابان 22 بهمن


now we can take a look at new data that show how many banks we have in each street

In [9]:
ndf['address'].value_counts()

خیابان امام خمینی                                                 13
خیابان شهید بهشتی                                                 10
خیابان دانشگاه                                                     9
خیابان امام خمینی(ره)                                              8
خیابان مشهد                                                        7
                                                                  ..
ميدان شهداء -  ابتداي  خیابان  شهيد رجايي - جنب سينما عصر جديد     1
كيلومتر 5 جاده اراك تهران                                          1
خیابان رودکی                                                       1
خیابان دكتر بهشتي                                                  1
میدان فرمانداری                                                    1
Name: address, Length: 121, dtype: int64

no we must remove space between addresses to remove repetitive results
then add new column 'address_1' to data frame

In [10]:
 ndf['address_1'] = ndf['address'].apply(lambda x: x.replace(' ',''))

In [11]:
ndf['address_1']

0            خيابانامامخميني
1             خیابانامیرکبیر
2      خیابانقائممقامفراهانی
3        خیابانشهیددکتربهشتی
4         خیابانمیرزایشیرازی
               ...          
207              خیابانمحسنی
208                میدانشهدا
209          خیابانشهیدرجایی
210               خیابانامام
211             خیابان22بهمن
Name: address_1, Length: 212, dtype: object

now add city name front of each street name for location finding improvement

In [12]:
ndf['address_1'] = ndf['address_1'].add(',اراک')

In [13]:
ndf['address_1'].to_excel('address_1.xlsx',sheet_name='Sheet1')

In [14]:
ndf

,نام,کد شعبه,نوع فعالیت,کد اختصاصی,بانک,استان,شهر,آدرس,تلفن,نوع (شعبه/باجه),address,address_1
0,اراک,114,ریالی - ارزی,1100114,بانک صنعت و معدن,مرکزی,اراک,خيابان امام خميني، روبروي 12متري ميراشرفي,08634774740,شعبه,خيابان امام خميني,"خيابانامامخميني,اراک"
1,خیابان امیرکبیر اراک,88401,ریالی,1288401,بانک ملت,مرکزی,اراک,خیابان امیر کبیر,08632233452,شعبه,خیابان امیر کبیر,"خیابانامیرکبیر,اراک"
2,خیابان قائم مقام فراهانی اراک,88427,ریالی,1288427,بانک ملت,مرکزی,اراک,خیابان قائم مقام فراهانی، نبش کوچه زرین,08632232831,شعبه,خیابان قائم مقام فراهانی,"خیابانقائممقامفراهانی,اراک"
3,خیابان شهید بهشتی اراک,88435,ریالی,1288435,بانک ملت,مرکزی,اراک,خیابان شهید دکتر بهشتی، بالاتر از میدان هفت تیر,08632221730,شعبه,خیابان شهید دکتر بهشتی,"خیابانشهیددکتربهشتی,اراک"
4,بیمه ایران,88435,NaN,1288435,بانک ملت,مرکزی,اراک,خیابان میرزای شیرازی، نرسیده به پل برق، ساختما...,NaN,باجه,خیابان میرزای شیرازی,"خیابانمیرزایشیرازی,اراک"
...,...,...,...,...,...,...,...,...,...,...,...,...
207,مرکزی اراک,3701,ریالی,7303701,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,خیابان محسنی، نبش گذر مسجد حاج تقی خان,08632215934,شعبه,خیابان محسنی,"خیابانمحسنی,اراک"
208,شهدا,3702,ریالی,7303702,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,میدان شهدا,08632236623,شعبه,میدان شهدا,"میدانشهدا,اراک"
209,مرکزی- اراک,3912,ریالی,8003912,موسسه اعتباری غیربانکی نور,مرکزی,اراک,خیابان شهید رجایی، نبش کوچه ابطحی,08632239866,شعبه,خیابان شهید رجایی,"خیابانشهیدرجایی,اراک"
210,امام خمینی- اراک,3913,ریالی,8003913,موسسه اعتباری غیربانکی نور,مرکزی,اراک,خیابان امام، نبش کوچه حاج علیخانی,08632218320,شعبه,خیابان امام,"خیابانامام,اراک"


prepare address

In [15]:
ndf['آدرس'] = ndf['آدرس'].apply(lambda x:  'اراک,' + x)

In [16]:
ndf

,نام,کد شعبه,نوع فعالیت,کد اختصاصی,بانک,استان,شهر,آدرس,تلفن,نوع (شعبه/باجه),address,address_1
0,اراک,114,ریالی - ارزی,1100114,بانک صنعت و معدن,مرکزی,اراک,"اراک,خيابان امام خميني، روبروي 12متري ميراشرفي",08634774740,شعبه,خيابان امام خميني,"خيابانامامخميني,اراک"
1,خیابان امیرکبیر اراک,88401,ریالی,1288401,بانک ملت,مرکزی,اراک,"اراک,خیابان امیر کبیر",08632233452,شعبه,خیابان امیر کبیر,"خیابانامیرکبیر,اراک"
2,خیابان قائم مقام فراهانی اراک,88427,ریالی,1288427,بانک ملت,مرکزی,اراک,"اراک,خیابان قائم مقام فراهانی، نبش کوچه زرین",08632232831,شعبه,خیابان قائم مقام فراهانی,"خیابانقائممقامفراهانی,اراک"
3,خیابان شهید بهشتی اراک,88435,ریالی,1288435,بانک ملت,مرکزی,اراک,"اراک,خیابان شهید دکتر بهشتی، بالاتر از میدان ه...",08632221730,شعبه,خیابان شهید دکتر بهشتی,"خیابانشهیددکتربهشتی,اراک"
4,بیمه ایران,88435,NaN,1288435,بانک ملت,مرکزی,اراک,"اراک,خیابان میرزای شیرازی، نرسیده به پل برق، س...",NaN,باجه,خیابان میرزای شیرازی,"خیابانمیرزایشیرازی,اراک"
...,...,...,...,...,...,...,...,...,...,...,...,...
207,مرکزی اراک,3701,ریالی,7303701,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,"اراک,خیابان محسنی، نبش گذر مسجد حاج تقی خان",08632215934,شعبه,خیابان محسنی,"خیابانمحسنی,اراک"
208,شهدا,3702,ریالی,7303702,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,"اراک,میدان شهدا",08632236623,شعبه,میدان شهدا,"میدانشهدا,اراک"
209,مرکزی- اراک,3912,ریالی,8003912,موسسه اعتباری غیربانکی نور,مرکزی,اراک,"اراک,خیابان شهید رجایی، نبش کوچه ابطحی",08632239866,شعبه,خیابان شهید رجایی,"خیابانشهیدرجایی,اراک"
210,امام خمینی- اراک,3913,ریالی,8003913,موسسه اعتباری غیربانکی نور,مرکزی,اراک,"اراک,خیابان امام، نبش کوچه حاج علیخانی",08632218320,شعبه,خیابان امام,"خیابانامام,اراک"


now convert this csv to ecxel and start finding location in google map and add it to data frame!<br>
Nominatim couldnt find my location peroperly!<br>

In [17]:
ndf.to_csv(r'C:\Users\a.f.khojasteh\newreport_1.csv', index = False)

In [18]:
ndff = pd.read_csv('newreport_1.csv')

In [19]:
data=pd.read_csv('Report_22.csv')

### i added 'location' and 'banknames' from data collected from google map one by one in an coverted excell file 
### so these two columns doesnt match whole dataframe columns
### the i decide to covert them to a new dataframe for ploting 

In [20]:
data

,نام,کد شعبه,نوع فعالیت,کد اختصاصی,بانک,استان,شهر,آدرس,bank names,location
0,اراک,114,ریالی - ارزی,1100114,بانک صنعت و معدن,مرکزی,اراک,خيابان امام خميني، روبروي 12متري ميراشرفي,pasargad bank,"34.083790, 49.685149"
1,خیابان امیرکبیر اراک,88401,ریالی,1288401,بانک ملت,مرکزی,اراک,خیابان امیر کبیر,pasargad bank,"34.086353, 49.688380"
2,خیابان قائم مقام فراهانی اراک,88427,ریالی,1288427,بانک ملت,مرکزی,اراک,خیابان قائم مقام فراهانی، نبش کوچه زرین,pasargad bank,"34.086400, 49.688367"
3,خیابان شهید بهشتی اراک,88435,ریالی,1288435,بانک ملت,مرکزی,اراک,خیابان شهید دکتر بهشتی، بالاتر از میدان هفت تیر,tose'e ta'avon bank,"34.086589, 49.701540"
4,بیمه ایران,88435,NaN,1288435,بانک ملت,مرکزی,اراک,خیابان میرزای شیرازی، نرسیده به پل برق، ساختما...,tose'e ta'avon bank,"34.087344, 49.701563"
...,...,...,...,...,...,...,...,...,...,...
202,مرکزی اراک,3701,ریالی,7303701,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,خیابان محسنی، نبش گذر مسجد حاج تقی خان,NaN,NaN
203,شهدا,3702,ریالی,7303702,مؤسسه اعتباری غیربانکی كوثر مرکزی,مرکزی,اراک,میدان شهدا,NaN,NaN
204,مرکزی- اراک,3912,ریالی,8003912,موسسه اعتباری غیربانکی نور,مرکزی,اراک,خیابان شهید رجایی، نبش کوچه ابطحی,NaN,NaN
205,امام خمینی- اراک,3913,ریالی,8003913,موسسه اعتباری غیربانکی نور,مرکزی,اراک,خیابان امام، نبش کوچه حاج علیخانی,NaN,NaN


In [21]:
base = pd.DataFrame(data = data,columns=['bank names','location'])
base.dropna(inplace=True)
base

,bank names,location
0,pasargad bank,"34.083790, 49.685149"
1,pasargad bank,"34.086353, 49.688380"
2,pasargad bank,"34.086400, 49.688367"
3,tose'e ta'avon bank,"34.086589, 49.701540"
4,tose'e ta'avon bank,"34.087344, 49.701563"
...,...,...
197,ghavamin bank,"34.112511, 49.694347"
198,ghavamin bank,"34.122478, 49.692303"
199,ghavamin bank,"34.120612, 49.697456"
200,ghavamin bank,"34.125944, 49.701738"


In [22]:
base['lat'] = base['location'].apply(lambda x: x.split(',')[0])
base['long'] = base['location'].apply(lambda x: x.split(',')[1])
base['banknames'] = base['bank names']
base.drop(['bank names'],axis = 1,inplace=True)

### convert object type to numeric

In [23]:
base['nlat'] = pd.to_numeric(base['lat'])
base['nlong'] = pd.to_numeric(base['long'])
base.drop(['lat','long'],axis=1,inplace=True)
base['lat'] = base['nlat']
base['long'] = base['nlong']
base.drop(['nlat','nlong'],axis=1,inplace=True)
base

,location,banknames,lat,long
0,"34.083790, 49.685149",pasargad bank,34.083790,49.685149
1,"34.086353, 49.688380",pasargad bank,34.086353,49.688380
2,"34.086400, 49.688367",pasargad bank,34.086400,49.688367
3,"34.086589, 49.701540",tose'e ta'avon bank,34.086589,49.701540
4,"34.087344, 49.701563",tose'e ta'avon bank,34.087344,49.701563
...,...,...,...,...
197,"34.112511, 49.694347",ghavamin bank,34.112511,49.694347
198,"34.122478, 49.692303",ghavamin bank,34.122478,49.692303
199,"34.120612, 49.697456",ghavamin bank,34.120612,49.697456
200,"34.125944, 49.701738",ghavamin bank,34.125944,49.701738


In [24]:
latitude = base['lat']
longitude = base['long']

### find restaurants in foursquare service

In [25]:
search_query = 'restaurant'
radius = 8000
address ='arak'
geolocator = Nominatim(user_agent="foursquare_agent",domain='nominatim.openstreetmap.org')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
CLIENT_ID = 'ZAOPF1M04XRFHHMB0CAX0Z5XNWVKQPSB1STBTV0YXSW2V2O4' # your Foursquare ID
CLIENT_SECRET = 'IEUBYFQ0HHVWIV2SZ5JP4M25WS5CED0ZMHPURHS3PIUIUDBK' # your Foursquare Secret
VERSION = '20200602'
LIMIT = 3000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f576e3fbf4f711750fbfb73'},
 'response': {'venues': [{'id': '58bed7fe13af1c3e8420a7a5',
    'name': 'Fandogh Café & Restaurant | کافه و رستوران فندق (کافه و رستوران فندق)',
    'location': {'address': 'Imami Aly., Abbas Abad St.',
     'lat': 34.085705,
     'lng': 49.689064,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.085705,
       'lng': 49.689064}],
     'distance': 93,
     'cc': 'IR',
     'city': 'اراک',
     'state': 'مرکزی',
     'country': 'ایران',
     'formattedAddress': ['Imami Aly., Abbas Abad St.',
      'اراک, مرکزی',
      'ایران']},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1599565661',
    'hasPerk': False},
   {'id': '50bf2799e4b0502ae0c366f6',
    'name':

In [26]:
sase = results['response']['venues']
sase

[{'id': '58bed7fe13af1c3e8420a7a5',
  'name': 'Fandogh Café & Restaurant | کافه و رستوران فندق (کافه و رستوران فندق)',
  'location': {'address': 'Imami Aly., Abbas Abad St.',
   'lat': 34.085705,
   'lng': 49.689064,
   'labeledLatLngs': [{'label': 'display',
     'lat': 34.085705,
     'lng': 49.689064}],
   'distance': 93,
   'cc': 'IR',
   'city': 'اراک',
   'state': 'مرکزی',
   'country': 'ایران',
   'formattedAddress': ['Imami Aly., Abbas Abad St.', 'اراک, مرکزی', 'ایران']},
  'categories': [{'id': '4bf58dd8d48988d1c4941735',
    'name': 'Restaurant',
    'pluralName': 'Restaurants',
    'shortName': 'Restaurant',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1599565661',
  'hasPerk': False},
 {'id': '50bf2799e4b0502ae0c366f6',
  'name': 'Hattam Restaurant (رستوران حاتم)',
  'location': {'address': 'Abbas Abad St.',
   'lat': 34.084156503760184,
   'lng': 49.68616247177124,
   'label

In [27]:
dataframe = json_normalize(sase)
dataframe

C:\Users\a.f.khojasteh\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,58bed7fe13af1c3e8420a7a5,Fandogh Café & Restaurant | کافه و رستوران فند...,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1599565661,False,"Imami Aly., Abbas Abad St.",34.085705,49.689064,"[{'label': 'display', 'lat': 34.085705, 'lng':...",93,IR,اراک,مرکزی,ایران,"[Imami Aly., Abbas Abad St., اراک, مرکزی, ایران]",NaN,NaN
1,50bf2799e4b0502ae0c366f6,Hattam Restaurant (رستوران حاتم),"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1599565661,False,Abbas Abad St.,34.084157,49.686162,"[{'label': 'display', 'lat': 34.08415650376018...",360,IR,اراک,مرکزی,ایران,"[Abbas Abad St., اراک, مرکزی, ایران]",NaN,NaN
2,504f6b7dc84ca785313e0f7c,Samiei Restaurant | رستوران سمیعی (رستوران سمیعی),"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1599565661,False,Imam St.,34.089399,49.685476,"[{'label': 'display', 'lat': 34.08939895550417...",446,IR,اراک,مرکزی,ایران,"[Imam St. (at Shoura Sq.), اراک, مرکزی, ایران]",at Shoura Sq.,NaN
3,58ec29022bc5e241f171de3e,Shaan Restaurant & Café | کافه رستوران شان (کا...,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1599565661,False,Bist-o Dovom-e Bahman St.,34.084867,49.683287,"[{'label': 'display', 'lat': 34.08486736369742...",544,IR,اراک,مرکزی,ایران,"[Bist-o Dovom-e Bahman St. (at Najimi Aly.), ا...",at Najimi Aly.,NaN
4,51596428e4b0ddfa36307e6c,Parsian Restaurant | مجتمع پذیرایی پارسیان (مج...,"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1599565661,False,Arak Bazaar,34.090732,49.692535,"[{'label': 'display', 'lat': 34.09073173046418...",579,IR,اراک,مرکزی,ایران,"[Arak Bazaar (Mohseni St.), اراک, مرکزی, ایران]",Mohseni St.,NaN
5,50e84e75582fd1ade3355bdc,Zeytoon Restaurant (رستوران زیتون),"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1599565661,False,Imam St.,34.090287,49.683609,"[{'label': 'display', 'lat': 34.09028747447554...",639,IR,اراک,مرکزی,ایران,"[Imam St., اراک, مرکزی, ایران]",NaN,NaN
6,537d1767498e0b5c15b58a47,Chaarsoogh Restaurant (سفره‌خانه سنتی چارسوق),"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1599565661,False,Arak Bazaar,34.091545,49.692029,"[{'label': 'display', 'lat': 34.091545, 'lng':...",631,IR,اراک,مرکزی,ایران,"[Arak Bazaar, اراک, مرکزی, ایران]",NaN,NaN
7,52fb92ab498ec3d58899ad4d,Farahzad Restaurant (سفره‌خانه سنتی فرحزاد),"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1599565661,False,Hefdah Metri St.,34.087367,49.696190,"[{'label': 'display', 'lat': 34.08736694625045...",684,IR,اراک,مرکزی,ایران,"[Hefdah Metri St. (Imam St.), اراک, مرکزی, ایران]",Imam St.,NaN
8,535fde7b498ecab0a45b0125,Toranj Restaurant (غذاکده سنتی ترنج),"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1599565661,False,Dr. Hessabi Xrd.,34.085762,49.696898,"[{'label': 'display', 'lat': 34.08576193502242...",747,IR,اراک,مرکزی,ایران,"[Dr. Hessabi Xrd. (Hefdah Metri St.), اراک, مر...",Hefdah Metri St.,NaN
9,574ec66e498e412ad6eb7eec,Hojreh Restaurant | سفره خانه سنتی حجره (سفره ...,"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1599565661,False,"Nabati Aly., Mohseni St.",34.092384,49.693372,"[{'label': 'display', 'lat': 34.09238434227428...",774,IR,اراک,مرکزی,ایران,"[Nabati Aly., Mohseni St., اراک, مرکزی, ایران]",NaN,NaN


### restaurant dataframe from foursquare

In [28]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Fandogh Café & Restaurant | کافه و رستوران فند...,Restaurant,"Imami Aly., Abbas Abad St.",34.085705,49.689064,"[{'label': 'display', 'lat': 34.085705, 'lng':...",93,IR,اراک,مرکزی,ایران,"[Imami Aly., Abbas Abad St., اراک, مرکزی, ایران]",NaN,NaN,58bed7fe13af1c3e8420a7a5
1,Hattam Restaurant (رستوران حاتم),Persian Restaurant,Abbas Abad St.,34.084157,49.686162,"[{'label': 'display', 'lat': 34.08415650376018...",360,IR,اراک,مرکزی,ایران,"[Abbas Abad St., اراک, مرکزی, ایران]",NaN,NaN,50bf2799e4b0502ae0c366f6
2,Samiei Restaurant | رستوران سمیعی (رستوران سمیعی),Persian Restaurant,Imam St.,34.089399,49.685476,"[{'label': 'display', 'lat': 34.08939895550417...",446,IR,اراک,مرکزی,ایران,"[Imam St. (at Shoura Sq.), اراک, مرکزی, ایران]",at Shoura Sq.,NaN,504f6b7dc84ca785313e0f7c
3,Shaan Restaurant & Café | کافه رستوران شان (کا...,Diner,Bist-o Dovom-e Bahman St.,34.084867,49.683287,"[{'label': 'display', 'lat': 34.08486736369742...",544,IR,اراک,مرکزی,ایران,"[Bist-o Dovom-e Bahman St. (at Najimi Aly.), ا...",at Najimi Aly.,NaN,58ec29022bc5e241f171de3e
4,Parsian Restaurant | مجتمع پذیرایی پارسیان (مج...,Persian Restaurant,Arak Bazaar,34.090732,49.692535,"[{'label': 'display', 'lat': 34.09073173046418...",579,IR,اراک,مرکزی,ایران,"[Arak Bazaar (Mohseni St.), اراک, مرکزی, ایران]",Mohseni St.,NaN,51596428e4b0ddfa36307e6c
5,Zeytoon Restaurant (رستوران زیتون),Restaurant,Imam St.,34.090287,49.683609,"[{'label': 'display', 'lat': 34.09028747447554...",639,IR,اراک,مرکزی,ایران,"[Imam St., اراک, مرکزی, ایران]",NaN,NaN,50e84e75582fd1ade3355bdc
6,Chaarsoogh Restaurant (سفره‌خانه سنتی چارسوق),Persian Restaurant,Arak Bazaar,34.091545,49.692029,"[{'label': 'display', 'lat': 34.091545, 'lng':...",631,IR,اراک,مرکزی,ایران,"[Arak Bazaar, اراک, مرکزی, ایران]",NaN,NaN,537d1767498e0b5c15b58a47
7,Farahzad Restaurant (سفره‌خانه سنتی فرحزاد),Persian Restaurant,Hefdah Metri St.,34.087367,49.696190,"[{'label': 'display', 'lat': 34.08736694625045...",684,IR,اراک,مرکزی,ایران,"[Hefdah Metri St. (Imam St.), اراک, مرکزی, ایران]",Imam St.,NaN,52fb92ab498ec3d58899ad4d
8,Toranj Restaurant (غذاکده سنتی ترنج),Persian Restaurant,Dr. Hessabi Xrd.,34.085762,49.696898,"[{'label': 'display', 'lat': 34.08576193502242...",747,IR,اراک,مرکزی,ایران,"[Dr. Hessabi Xrd. (Hefdah Metri St.), اراک, مر...",Hefdah Metri St.,NaN,535fde7b498ecab0a45b0125
9,Hojreh Restaurant | سفره خانه سنتی حجره (سفره ...,Persian Restaurant,"Nabati Aly., Mohseni St.",34.092384,49.693372,"[{'label': 'display', 'lat': 34.09238434227428...",774,IR,اراک,مرکزی,ایران,"[Nabati Aly., Mohseni St., اراک, مرکزی, ایران]",NaN,NaN,574ec66e498e412ad6eb7eec


In [29]:
search_query = 'bank'
radius = 8000
address ='arak'
geolocator = Nominatim(user_agent="foursquare_agent",domain='nominatim.openstreetmap.org')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
CLIENT_ID = 'ZAOPF1M04XRFHHMB0CAX0Z5XNWVKQPSB1STBTV0YXSW2V2O4' # your Foursquare ID
CLIENT_SECRET = 'IEUBYFQ0HHVWIV2SZ5JP4M25WS5CED0ZMHPURHS3PIUIUDBK' # your Foursquare Secret
VERSION = '20200602'
LIMIT = 3000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5f576f1abeb40e3387866be9'},
 'response': {'venues': [{'id': '504f2f061648474837a3a624',
    'name': 'Passargad Bank (بانک پاسارگاد)',
    'location': {'address': 'Bagh Melli',
     'lat': 34.086155782122624,
     'lng': 49.68860864639282,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.086155782122624,
       'lng': 49.68860864639282}],
     'distance': 45,
     'cc': 'IR',
     'city': 'اراک',
     'state': 'مرکزی',
     'country': 'ایران',
     'formattedAddress': ['Bagh Melli', 'اراک, مرکزی', 'ایران']},
    'categories': [{'id': '4bf58dd8d48988d10a951735',
      'name': 'Bank',
      'pluralName': 'Banks',
      'shortName': 'Bank',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1599565664',
    'hasPerk': False},
   {'id': '50e93500e4b0d6d8b60a9409',
    'name': 'Saderat Bank | بانک صادرات شعبه باغ ملی (بانک صادرات شعبه باغ 

In [30]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5f576ef27f58eb2f2f1ff308'},
 'response': {'venues': [{'id': '504f2f061648474837a3a624',
    'name': 'Passargad Bank (بانک پاسارگاد)',
    'location': {'address': 'Bagh Melli',
     'lat': 34.086155782122624,
     'lng': 49.68860864639282,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.086155782122624,
       'lng': 49.68860864639282}],
     'distance': 45,
     'cc': 'IR',
     'city': 'اراک',
     'state': 'مرکزی',
     'country': 'ایران',
     'formattedAddress': ['Bagh Melli', 'اراک, مرکزی', 'ایران']},
    'categories': [{'id': '4bf58dd8d48988d10a951735',
      'name': 'Bank',
      'pluralName': 'Banks',
      'shortName': 'Bank',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1599565665',
    'hasPerk': False},
   {'id': '50e93500e4b0d6d8b60a9409',
    'name': 'Saderat Bank | بانک صادرات شعبه باغ ملی (بانک صادرات شعبه باغ 

In [31]:
venue = result['response']['venues']
venue

[{'id': '504f2f061648474837a3a624',
  'name': 'Passargad Bank (بانک پاسارگاد)',
  'location': {'address': 'Bagh Melli',
   'lat': 34.086155782122624,
   'lng': 49.68860864639282,
   'labeledLatLngs': [{'label': 'display',
     'lat': 34.086155782122624,
     'lng': 49.68860864639282}],
   'distance': 45,
   'cc': 'IR',
   'city': 'اراک',
   'state': 'مرکزی',
   'country': 'ایران',
   'formattedAddress': ['Bagh Melli', 'اراک, مرکزی', 'ایران']},
  'categories': [{'id': '4bf58dd8d48988d10a951735',
    'name': 'Bank',
    'pluralName': 'Banks',
    'shortName': 'Bank',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1599565665',
  'hasPerk': False},
 {'id': '50e93500e4b0d6d8b60a9409',
  'name': 'Saderat Bank | بانک صادرات شعبه باغ ملی (بانک صادرات شعبه باغ ملی)',
  'location': {'address': 'Bagh Melli',
   'lat': 34.0862135383515,
   'lng': 49.68849062919617,
   'labeledLatLngs': [{'label': '

In [32]:
data_1 = json_normalize(venue)
data_1

C:\Users\a.f.khojasteh\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,504f2f061648474837a3a624,Passargad Bank (بانک پاسارگاد),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Bagh Melli,34.086156,49.688609,"[{'label': 'display', 'lat': 34.08615578212262...",45,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN
1,50e93500e4b0d6d8b60a9409,Saderat Bank | بانک صادرات شعبه باغ ملی (بانک ...,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Bagh Melli,34.086214,49.688491,"[{'label': 'display', 'lat': 34.0862135383515,...",47,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN
2,51ff57d4498e88312d4173cd,Sepah Bank (بانک سپه),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Bagh Melli,34.086296,49.689550,"[{'label': 'display', 'lat': 34.08629556573898...",69,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN
3,575ba614498ece0dbc0daf89,Melli Bank | بانک ملی شعبه مرکزی (بانک ملی شعب...,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Malek St.,34.085694,49.690561,"[{'label': 'display', 'lat': 34.08569373087315...",183,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN
4,51c29635498e903cfe502ded,Sepah Bank (بانک سپه),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Malek St.,34.084648,49.691364,"[{'label': 'display', 'lat': 34.08464813232422...",312,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN
5,52f4a6c011d216757d4307ea,Keshavarazi Bank (بانک کشاورزی),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Hashemi Nejad St.,34.084958,49.684116,"[{'label': 'display', 'lat': 34.08495783543008...",469,IR,اراک,مرکزی,ایران,"[Hashemi Nejad St., اراک, مرکزی, ایران]",NaN
6,52dcec12498ec31a804b28d3,Post Bank (پست بانک),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,"Hefdah Metri St, Malek St.",34.084614,49.693984,"[{'label': 'display', 'lat': 34.08461412052909...",519,IR,اراک,مرکزی,ایران,"[Hefdah Metri St, Malek St., اراک, مرکزی, ایران]",NaN
7,5861f97c076be150f627bf31,Tejarat Bank HQ | سرپرستی شعب بانک تجارت استان...,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Imam Ave.,34.089842,49.684625,"[{'label': 'display', 'lat': 34.089842, 'lng':...",536,IR,اراک,مرکزی,ایران,"[Imam Ave., اراک, مرکزی, ایران]",NaN
8,502e2614011c4f40e04beb99,Saman Bank | بانک سامان (بانک سامان),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Malek St.,34.083368,49.693388,"[{'label': 'display', 'lat': 34.08336769623738...",546,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN
9,527215af11d246e82e9e9ad6,Saderat Bank | بانک صادرات شعبه ملک (بانک صادر...,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1599565665,False,Malek St.,34.083072,49.693448,"[{'label': 'display', 'lat': 34.08307218798224...",572,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN


In [33]:
filtered_columns = ['name', 'categories'] + [col for col in data_1.columns if col.startswith('location.')] + ['id']
datafilter = data_1.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
datafilter['categories'] = datafilter.apply(get_category_type, axis=1)

# clean column names by keeping only last term
datafilter.columns = [column.split('.')[-1] for column in datafilter.columns]

datafilter

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,id
0,Passargad Bank (بانک پاسارگاد),Bank,Bagh Melli,34.086156,49.688609,"[{'label': 'display', 'lat': 34.08615578212262...",45,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN,504f2f061648474837a3a624
1,Saderat Bank | بانک صادرات شعبه باغ ملی (بانک ...,Bank,Bagh Melli,34.086214,49.688491,"[{'label': 'display', 'lat': 34.0862135383515,...",47,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN,50e93500e4b0d6d8b60a9409
2,Sepah Bank (بانک سپه),Bank,Bagh Melli,34.086296,49.689550,"[{'label': 'display', 'lat': 34.08629556573898...",69,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN,51ff57d4498e88312d4173cd
3,Melli Bank | بانک ملی شعبه مرکزی (بانک ملی شعب...,Bank,Malek St.,34.085694,49.690561,"[{'label': 'display', 'lat': 34.08569373087315...",183,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,575ba614498ece0dbc0daf89
4,Sepah Bank (بانک سپه),Bank,Malek St.,34.084648,49.691364,"[{'label': 'display', 'lat': 34.08464813232422...",312,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,51c29635498e903cfe502ded
5,Keshavarazi Bank (بانک کشاورزی),Bank,Hashemi Nejad St.,34.084958,49.684116,"[{'label': 'display', 'lat': 34.08495783543008...",469,IR,اراک,مرکزی,ایران,"[Hashemi Nejad St., اراک, مرکزی, ایران]",NaN,52f4a6c011d216757d4307ea
6,Post Bank (پست بانک),Bank,"Hefdah Metri St, Malek St.",34.084614,49.693984,"[{'label': 'display', 'lat': 34.08461412052909...",519,IR,اراک,مرکزی,ایران,"[Hefdah Metri St, Malek St., اراک, مرکزی, ایران]",NaN,52dcec12498ec31a804b28d3
7,Tejarat Bank HQ | سرپرستی شعب بانک تجارت استان...,Bank,Imam Ave.,34.089842,49.684625,"[{'label': 'display', 'lat': 34.089842, 'lng':...",536,IR,اراک,مرکزی,ایران,"[Imam Ave., اراک, مرکزی, ایران]",NaN,5861f97c076be150f627bf31
8,Saman Bank | بانک سامان (بانک سامان),Bank,Malek St.,34.083368,49.693388,"[{'label': 'display', 'lat': 34.08336769623738...",546,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,502e2614011c4f40e04beb99
9,Saderat Bank | بانک صادرات شعبه ملک (بانک صادر...,Bank,Malek St.,34.083072,49.693448,"[{'label': 'display', 'lat': 34.08307218798224...",572,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,527215af11d246e82e9e9ad6


### bank dataframe from foursquare

In [34]:
datafilter

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,id
0,Passargad Bank (بانک پاسارگاد),Bank,Bagh Melli,34.086156,49.688609,"[{'label': 'display', 'lat': 34.08615578212262...",45,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN,504f2f061648474837a3a624
1,Saderat Bank | بانک صادرات شعبه باغ ملی (بانک ...,Bank,Bagh Melli,34.086214,49.688491,"[{'label': 'display', 'lat': 34.0862135383515,...",47,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN,50e93500e4b0d6d8b60a9409
2,Sepah Bank (بانک سپه),Bank,Bagh Melli,34.086296,49.689550,"[{'label': 'display', 'lat': 34.08629556573898...",69,IR,اراک,مرکزی,ایران,"[Bagh Melli, اراک, مرکزی, ایران]",NaN,51ff57d4498e88312d4173cd
3,Melli Bank | بانک ملی شعبه مرکزی (بانک ملی شعب...,Bank,Malek St.,34.085694,49.690561,"[{'label': 'display', 'lat': 34.08569373087315...",183,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,575ba614498ece0dbc0daf89
4,Sepah Bank (بانک سپه),Bank,Malek St.,34.084648,49.691364,"[{'label': 'display', 'lat': 34.08464813232422...",312,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,51c29635498e903cfe502ded
5,Keshavarazi Bank (بانک کشاورزی),Bank,Hashemi Nejad St.,34.084958,49.684116,"[{'label': 'display', 'lat': 34.08495783543008...",469,IR,اراک,مرکزی,ایران,"[Hashemi Nejad St., اراک, مرکزی, ایران]",NaN,52f4a6c011d216757d4307ea
6,Post Bank (پست بانک),Bank,"Hefdah Metri St, Malek St.",34.084614,49.693984,"[{'label': 'display', 'lat': 34.08461412052909...",519,IR,اراک,مرکزی,ایران,"[Hefdah Metri St, Malek St., اراک, مرکزی, ایران]",NaN,52dcec12498ec31a804b28d3
7,Tejarat Bank HQ | سرپرستی شعب بانک تجارت استان...,Bank,Imam Ave.,34.089842,49.684625,"[{'label': 'display', 'lat': 34.089842, 'lng':...",536,IR,اراک,مرکزی,ایران,"[Imam Ave., اراک, مرکزی, ایران]",NaN,5861f97c076be150f627bf31
8,Saman Bank | بانک سامان (بانک سامان),Bank,Malek St.,34.083368,49.693388,"[{'label': 'display', 'lat': 34.08336769623738...",546,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,502e2614011c4f40e04beb99
9,Saderat Bank | بانک صادرات شعبه ملک (بانک صادر...,Bank,Malek St.,34.083072,49.693448,"[{'label': 'display', 'lat': 34.08307218798224...",572,IR,اراک,مرکزی,ایران,"[Malek St., اراک, مرکزی, ایران]",NaN,527215af11d246e82e9e9ad6


## banks and restaurants map in arak,iran

In [56]:
foursquare_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the center of arak city

# add a red circle marker to represent the center of Arak city
folium.features.CircleMarker(
    [latitude, longitude],
    radius=12,
    color='black',
    popup='Arak',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(foursquare_map)

# restaurant of foursquare
for lati, longi, labely in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lati, longi],
        radius=7,
        color='blue',
        popup=labely,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(foursquare_map)

# banks of foursquare
for lat, lng, label in zip(datafilter.lat, datafilter.lng, datafilter.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=7,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(foursquare_map)
# display map
foursquare_map

## so you can see number of banks equall to restaurants in a small city
was it so weired so far?! wait because foursqure data is not compelete
so i start collect real data from https://www.cbi.ir/BanksInstitutions/BankInstitute.aspx and add locations from google map to the file and save it to a new dataframe 'base'

## dataframe

In [36]:
base = pd.DataFrame(data = data,columns=['bank names','location'])
base.dropna(inplace=True)
base['lat'] = base['location'].apply(lambda x: x.split(',')[0])

base['long'] = base['location'].apply(lambda x: x.split(',')[1])

base['banknames'] = base['bank names']

base.drop(['bank names'],axis = 1,inplace=True)
base['nlat'] = pd.to_numeric(base['lat'])
base['nlong'] = pd.to_numeric(base['long'])
base.drop(['lat','long'],axis=1,inplace=True)
base['lat'] = base['nlat']
base['long'] = base['nlong']
base.drop(['nlat','nlong'],axis=1,inplace=True)

type has been corrected

In [37]:
base.lat

0      34.083790
1      34.086353
2      34.086400
3      34.086589
4      34.087344
         ...    
197    34.112511
198    34.122478
199    34.120612
200    34.125944
201    34.124773
Name: lat, Length: 202, dtype: float64

### visualize true number of banks collected from above reference and foursquare restaurants data 
#### real number of banks in compare with restaurants

In [57]:
handy_data_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the center of arak city

# add a red circle marker to represent the center of Arak city
folium.features.CircleMarker(
    [latitude, longitude],
    radius=12,
    color='red',
    popup='Arak',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(handy_data_map)

# banks of collecetd data
for lat, long  in zip(base['lat'], base['long'] ):
    folium.features.CircleMarker(
        [lat, long],
        radius=7,
        color='blue',
        popup='bank',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(handy_data_map)
# add the restaurants as green circle markers    
for lat_1, long_1  in zip(dataframe_filtered.lat, dataframe_filtered.lng ):
    folium.features.CircleMarker(
        [lat_1, long_1],
        radius=7,
        color='green',
        popup='restaurant',
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(handy_data_map)
# display map
handy_data_map

### so you can see that number of banks(blue circles) is more than number of restaurants(green circles)!

### data manupolation of 'ndf' that collected from 
https://www.cbi.ir/BanksInstitutions/BankInstitute.aspx 

In [39]:
ndf.drop(ndf.iloc[0:,0:4],axis=1,inplace=True)

In [40]:
ndf.drop(ndf.iloc[0:,5:7],axis=1,inplace=True)
ndf.drop(ndf.iloc[0:,8:10],axis=1,inplace=True)

In [41]:
ndf.drop([ 'استان ', 'شهر ', 'تلفن'],axis=1,inplace=True)
ndf

,بانک,آدرس,address_1
0,بانک صنعت و معدن,"اراک,خيابان امام خميني، روبروي 12متري ميراشرفي","خيابانامامخميني,اراک"
1,بانک ملت,"اراک,خیابان امیر کبیر","خیابانامیرکبیر,اراک"
2,بانک ملت,"اراک,خیابان قائم مقام فراهانی، نبش کوچه زرین","خیابانقائممقامفراهانی,اراک"
3,بانک ملت,"اراک,خیابان شهید دکتر بهشتی، بالاتر از میدان ه...","خیابانشهیددکتربهشتی,اراک"
4,بانک ملت,"اراک,خیابان میرزای شیرازی، نرسیده به پل برق، س...","خیابانمیرزایشیرازی,اراک"
...,...,...,...
207,مؤسسه اعتباری غیربانکی كوثر مرکزی,"اراک,خیابان محسنی، نبش گذر مسجد حاج تقی خان","خیابانمحسنی,اراک"
208,مؤسسه اعتباری غیربانکی كوثر مرکزی,"اراک,میدان شهدا","میدانشهدا,اراک"
209,موسسه اعتباری غیربانکی نور,"اراک,خیابان شهید رجایی، نبش کوچه ابطحی","خیابانشهیدرجایی,اراک"
210,موسسه اعتباری غیربانکی نور,"اراک,خیابان امام، نبش کوچه حاج علیخانی","خیابانامام,اراک"


**we dont need city name(اراک) above 'address_1' row**

In [42]:
ndf['address_1'] = ndf['address_1'].apply(lambda x: x.split(',')[0])
ndf['آدرس'] = ndf['آدرس'].apply(lambda x: x.split(',')[-1])
ndf['fixed_address'] = ndf['آدرس']
ndf['bank'] = ndf['بانک ']
ndf.drop(['آدرس'],axis = 1,inplace = True)
ndf.drop(['بانک '],axis=1,inplace=True)

In [43]:
del ndf['fixed_address']

In [44]:
ndf

,address_1,bank
0,خيابانامامخميني,بانک صنعت و معدن
1,خیابانامیرکبیر,بانک ملت
2,خیابانقائممقامفراهانی,بانک ملت
3,خیابانشهیددکتربهشتی,بانک ملت
4,خیابانمیرزایشیرازی,بانک ملت
...,...,...
207,خیابانمحسنی,مؤسسه اعتباری غیربانکی كوثر مرکزی
208,میدانشهدا,مؤسسه اعتباری غیربانکی كوثر مرکزی
209,خیابانشهیدرجایی,موسسه اعتباری غیربانکی نور
210,خیابانامام,موسسه اعتباری غیربانکی نور


In [45]:
ndf['address_1'].value_counts()

خیابانامامخمینی          14
خیابانشهیدبهشتی          10
خیاباندانشگاه            10
خیابانامامخمینی(ره)       9
خیابانمشهد                9
                         ..
ابتدایخیابانشهیدرجایی     1
میدانقرآن                 1
جادهکمربندیاراک           1
خیابانهفتادودوتن          1
خیابانشهیدطالقانی         1
Name: address_1, Length: 114, dtype: int64

In [46]:
ndf['bank'].value_counts() 

بانک ملی ایران                       28
بانک سپه                             25
بانک صادرات ایران                    20
بانک ملت                             18
بانک تجارت                           16
بانک کشاورزی                         14
بانک مسکن                            14
بانک انصار                           13
بانک رفاه کارگران                     9
بانک قوامین                           9
بانک قرض‌الحسنه مهر ایران             7
بانک توسعه تعاون                      4
بانک قرض‌الحسنه رسالت                 4
مؤسسه اعتباری غیربانکی ملل            3
موسسه اعتباری غیربانکی نور            3
بانک سینا                             2
مؤسسه اعتباری غیربانکی كوثر مرکزی     2
بانک اقتصادنوین                       2
بانک شهر                              2
بانک ایران زمین                       2
پست بانک ایران                        2
بانک پارسیان                          2
بانک پاسارگاد                         2
بانک سرمایه                           1
بانک سامان                            1


In [47]:
ndf

,address_1,bank
0,خيابانامامخميني,بانک صنعت و معدن
1,خیابانامیرکبیر,بانک ملت
2,خیابانقائممقامفراهانی,بانک ملت
3,خیابانشهیددکتربهشتی,بانک ملت
4,خیابانمیرزایشیرازی,بانک ملت
...,...,...
207,خیابانمحسنی,مؤسسه اعتباری غیربانکی كوثر مرکزی
208,میدانشهدا,مؤسسه اعتباری غیربانکی كوثر مرکزی
209,خیابانشهیدرجایی,موسسه اعتباری غیربانکی نور
210,خیابانامام,موسسه اعتباری غیربانکی نور


## creat new dataframes from number of each columns 'banks' and 'address_1' 
**to visualize unique value of each categories**

In [48]:
newdf_1 = ndf['bank'].value_counts().rename_axis('sorted_banks').reset_index(name='counts')
newdf_1

,sorted_banks,counts
0,بانک ملی ایران,28
1,بانک سپه,25
2,بانک صادرات ایران,20
3,بانک ملت,18
4,بانک تجارت,16
5,بانک کشاورزی,14
6,بانک مسکن,14
7,بانک انصار,13
8,بانک رفاه کارگران,9
9,بانک قوامین,9


In [49]:
newdf_2 = ndf['address_1'].value_counts().rename_axis('street_address').reset_index(name='counts')
newdf_2

,street_address,counts
0,خیابانامامخمینی,14
1,خیابانشهیدبهشتی,10
2,خیاباندانشگاه,10
3,خیابانامامخمینی(ره),9
4,خیابانمشهد,9
...,...,...
109,ابتدایخیابانشهیدرجایی,1
110,میدانقرآن,1
111,جادهکمربندیاراک,1
112,خیابانهفتادودوتن,1


<h3>we will creat two dataframe for ploting one for "bank" column and another one for "address_1" column</h3>

In [50]:
newdf_1 = ndf['bank'].value_counts().rename_axis('sorted_banks').reset_index(name='counts')
newdf_1

,sorted_banks,counts
0,بانک ملی ایران,28
1,بانک سپه,25
2,بانک صادرات ایران,20
3,بانک ملت,18
4,بانک تجارت,16
5,بانک کشاورزی,14
6,بانک مسکن,14
7,بانک انصار,13
8,بانک رفاه کارگران,9
9,بانک قوامین,9


**newdf_2 manupolation**

In [51]:
newdf_2 = newdf_2.drop([4])

In [52]:
newdf_2

,street_address,counts
0,خیابانامامخمینی,14
1,خیابانشهیدبهشتی,10
2,خیاباندانشگاه,10
3,خیابانامامخمینی(ره),9
5,خیابانطالقانی,6
...,...,...
109,ابتدایخیابانشهیدرجایی,1
110,میدانقرآن,1
111,جادهکمربندیاراک,1
112,خیابانهفتادودوتن,1


In [53]:
newdf_2.reset_index(drop=True, inplace=True)

In [54]:
newdf_2.replace(newdf_2.iloc[0][1],23,inplace=True)

In [55]:
newdf_2.head(15)

,street_address,counts
0,خیابانامامخمینی,23
1,خیابانشهیدبهشتی,10
2,خیاباندانشگاه,10
3,خیابانامامخمینی(ره),9
4,خیابانطالقانی,6
5,میدانشهدا,5
6,خیابانشریعتی,5
7,خیابانقائممقامفراهانی,5
8,خیابانآیتالهغفاری,5
9,خيابانشهيدبهشتي,4


## Methodology <a name="methodology"></a>
* we collect data of banks in radius 3000 around arak city in foursquare in a json file
* and collect data of restaurants in the same way 
* so extract venues from json and save them to data frame by json_normalize()
* but i noted that data of banks that foursquare gave us is not complete because google map show more than 40 banks in arak,on the other hand i couldnt use google services because iran can not tranfer money according to US sanctions.so i must start collecting bank data and location from another source.So i decide to collect bank names and locations separately by using google map straight(that was the only way that i knew!)
* so i collected banks addresses from 'ndf' data frame
* and enter their location handy from google map one by one
* creat a new data frame 'base' with three columns : 1.location 2.lat(latitude) 3.long(longitude) 4.banknames
* but i couldn't match addresses in **'ndf'** and location in **'base'** so create new data frame for addresses that extract it from **'ndf'** with droping unnecessary columns then leave it with two columns :"address_1"(include the addresses of banks in persian that i remove spaces between words to find unique value of them and find how many banks is in each street) and "bank"(represent the bank name)
* at first i want to compare number of banks and restaurants in foursquare dataest to show unequality between these disribution
* then i want to show the audience real data that i collected in "base" dataset and compare the numbers with global standards
* and conclude that many of these branches must be closed in a economical crisis and pandemic

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's plot **distribution of restaurants and bankscollected from foursquare.com in a map**:

In [59]:
foursquare_map

**foursquare data couldn't gave me complete data of banks location however this data has enough insight to prove that bank branches has awfull distribution.And accorording to global standard determined by World Bank that tell us there were 12.5 bank branches for every 100,000 adults.ARAK city has 500,000 people including adults whitch for arak number of 12.5 multiple by 5 is equall to 62 bank branches  so at first glance there is normal number of them in arak but i want to show real number of banks in data collected by google map and compare it with number of restaurant that indicate as a normal economical activities :** 

In [60]:
handy_data_map

**also you can see irregular distribution of bank branches increasingly!we'll descuss about...**
**in this new data we recognize more than 200 bank branches that is 4 times more than standard indicated by World Bank and this is too extra ordinary!**

* **take a look at "newdf_2" dataframe that show how many bank branches are in each streets :**
* **And "newdf_1" dataframe that show how many banks are here in arak city**

In [61]:
newdf_2

,street_address,counts
0,خیابانامامخمینی,23
1,خیابانشهیدبهشتی,10
2,خیاباندانشگاه,10
3,خیابانامامخمینی(ره),9
4,خیابانطالقانی,6
...,...,...
108,ابتدایخیابانشهیدرجایی,1
109,میدانقرآن,1
110,جادهکمربندیاراک,1
111,خیابانهفتادودوتن,1


In [62]:
newdf_1

,sorted_banks,counts
0,بانک ملی ایران,28
1,بانک سپه,25
2,بانک صادرات ایران,20
3,بانک ملت,18
4,بانک تجارت,16
5,بانک کشاورزی,14
6,بانک مسکن,14
7,بانک انصار,13
8,بانک رفاه کارگران,9
9,بانک قوامین,9


* **each row represent a street and we have 23 bank branches in one street 'خیابان امام خمینی' , and so on
its so unusual**
* **also we have more than 30 bank in "newdf_1" data frame**
* **according to map more branches cluster at a small area and this show clearly big mistakes in distribution of branches**

## Results and Discussion <a name="results"></a>

**number of bank branches are unusually above the standards and i prove it by map and compare them to number of restaurants.it show us that we need bussinesses like restaurants more than inactive bank branches.on the other hand maps show us distribution of bank branches is so irregular and we could see that larg number of them group near few spots around center of city so i think we need no more bank branches.also we can use online banking automation systems instead.and if we take a look back at map more accurate we will notice that somewhere in city dont have enough even ATM's or bank branches and people must pass a long way to city center for their porpuses.**
### Now take a look at a part of an economical report published in "eghtesadonline" :
**"the numbers of bank branches indicate a subtle decline compared to the 23,549 branches reported for February, it is far above world standards.According to a World Bank report, there were 12.5 bank branches in the world for every 100,000 adults in 2016 while the same number for Iran in that year was 31.3. Iran’s average is also more than double the MENA average at 14.7 branches for every 100,000 adults.In comparison, figures for Turkey show it had 18.1 bank branches for the same number of people while the number for China was 8.8.If it wants to come near the global average, Iran should bring down the number of bank branches to 8,868, which means closing 11,730 branches.To reach the MENA average, a total of 10,169 branches should have been reduced to bring the total to 10,429 in 2016."[3]**

## Conclusion <a name="conclusion"></a>

Purpose of this report is show what number of bank branches and their distribution is irregular and unreasonably in the economy is in crisis.we have more than 30 banks in a small city and each person has an account in one or more of them,also they grouped a near area around city center.governers as first step must correct their distribution and in the second step must close at least half of them and they can merge banks together to save in cost and help economy get better.i think that what if bankers turn their usless branches into ordinary businesses like restaurant or technology centers or centers to group peoples with same level of thinking together for descussion and start learning new things. somewhere like universities without any protocol,because we dont have a specefic place to make together and coffies and restaurants are expensive and if you want to talk somebody about computer and seriouse problems you cant find comfort place like universities ecosystem(i wish join these active ecosystem sometime somewhere...)

# References :

* <a name="rf1">[1]</a>= https://www.cbi.ir/BanksInstitutions/BankInstitute.aspx 
* <a name="rf2">[2]</a>= https://www.google.com/maps
* <a name="rf3">[3]</a>= https://www.en.eghtesadonline.com/Section-economy-4/30257-iran-no-need-for-so-many-bank-branches
* <a name="rf6">[4]</a>= https://www.amar.org.ir
* <a name="rf5">[5]</a>= https://www.foursquare.com